## 前馈神经网络

前馈神经网络（feedforward neural network），也叫作多层感知机（MLP），是典型的深度学习模型。前馈网络的目的是近似某个函数 f^* 。例如，对于分类器， y=f^*(x) 将输入 x 映射到一个类别 y 。前馈网络定义了一个映射 y=f(x;t) ，并且学习参数t的值，使它能够得到最佳的函数近似。

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

# 数据集加载
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
sess = tf.InteractiveSession()

# 定义算法公式
in_units = 784
h1_units = 300
W1 = tf.Variable(tf.truncated_normal([in_units, h1_units], stddev=0.1))
b1 = tf.Variable(tf.zeros([h1_units]))
W2 = tf.Variable(tf.zeros([h1_units, 10]))
b2 = tf.Variable(tf.zeros([10]))

x = tf.placeholder(tf.float32, [None, in_units])
keep_prob = tf.placeholder(tf.float32)

hidden1 = tf.nn.relu(tf.matmul(x, W1) + b1)
hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
y = tf.nn.softmax(tf.matmul(hidden1_drop, W2) + b2)

# 定义损失函数和选择优化器来优化loss
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.AdagradOptimizer(0.3).minimize(cross_entropy)

# 训练,加入keep_prop作为计算图输入,并在训练时设为0.75,保留75%的节点
tf.global_variables_initializer().run()
for i in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    train_step.run({x: batch_xs, y_: batch_ys, keep_prob: 0.75})

# 对模型进行评测
prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

print("acc:", accuracy.eval({x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
acc: 0.9784


## 激活函数

激活函数是用来加入非线性因素的，解决线性模型所不能解决的问题

![激活函数](https://ws2.sinaimg.cn/large/006tKfTcgy1g0ywq50dkvj310h0ggmyv.jpg)

### sigmoid

sigmoid 是使用范围最广的一类激活函数，具有指数函数形状，它在物理意义上最为接近生物神经元。此外，(0, 1) 的输出还可以被表示作概率，或用于输入的归一化，代表性的如Sigmoid交叉熵损失函数。然而，sigmoid也有其自身的缺陷，最明显的就是饱和性。从上图可以看到，其两侧导数逐渐趋近于0 

### tanh

tanh也是一种非常常见的激活函数。与sigmoid相比，它的输出均值是0，使得其收敛速度要比sigmoid快，减少迭代次数。然而，从途中可以看出，tanh一样具有软饱和性，从而造成梯度消失。

### ReLU

ReLU的全称是Rectified Linear Units，是一种后来才出现的激活函数。 可以看到，当x<0时，ReLU硬饱和，而当x>0时，则不存在饱和问题。所以，ReLU 能够在x>0时保持梯度不衰减，从而缓解梯度消失问题。这让我们能够直接以监督的方式训练深度神经网络，而无需依赖无监督的逐层预训练。
然而，随着训练的推进，部分输入会落入硬饱和区，导致对应权重无法更新。这种现象被称为“神经元死亡”。与sigmoid类似，ReLU的输出均值也大于0，偏移现象和 神经元死亡会共同影响网络的收敛性。


## 深度学习正则化

### L1&L2 正则项

L1 和 L2 是最常见的正则化类型，它们会向代价函数中添加一个正则项，更新通用的代价函数。代价函数=损失（比如二元交叉熵）+正则项由于添加了正则项，权重矩阵的值会下降，因为它是假设神经网络具有更小的权重矩阵，从而使模型更简单。因此，这样也会大幅降低过拟合。

L1:
![L1](https://ws1.sinaimg.cn/large/006tKfTcgy1g0yx291ox0j309801fq2s.jpg)

L2:
![L2](https://ws3.sinaimg.cn/large/006tKfTcgy1g0yx33zzz3j3098019q2s.jpg)

## 深度学习中的优化

训练深度神经网络经常需要花费很多时间，在本文开头介绍了之前写的两篇文章，我们可以知道一般加快神经网络训练速度的方法有：合适的初始化策略，使用合适的激活函数；还可以使用批量归一化处理每层的输出数据，也可以使用预训练好的模型做迁移学习。这里我们介绍一些常用的优化算法，这能非常有效的缩短梯度下降所需要的时间，这些优化算法包括：SGD， Momentum optimization, Nesterov Accelerated Gradient, AdaGrad, RMSProp, and Adam optimization。

### SGD

![sgd](https://ws1.sinaimg.cn/large/006tKfTcly1g0yx5xc9z6j30k0071t9s.jpg)

### Momentun

动量可以加速学习，特别是处理高曲率、小但一致的梯度，或是带噪声的梯度。动量算法积累了之前梯度指数级衰减的移动平均，并且继续沿该方向移动。

![mon](https://ws3.sinaimg.cn/large/006tKfTcly1g0yx6uyfvoj30b10ad0td.jpg)

### AdaGrad

由于学习率对模型的性能有显著影响，又比较难以设置，于是有了自适应学习率算法。 AdaGrad算法，如下图所示，独立地适应所有模型参数的学习率，缩放每个参数反比于其所有梯度历史平方值总和的平方根。具有损失最大偏导的参数相应地有一个快速下降的学习率，而具有小偏导的参数在学习率上有相对较小的下降。净效果是在参数空间中更为平缓的倾斜方向会取得更大的进步。 然而，经验上已经发现，对于训练深度神经网络模型而言，从训练开始时积累梯度平方会导致有效学习率过早和过量的减小。AdaGrad 在某些深度学习模型上效果不错，但不是全部。

![ada](https://ws2.sinaimg.cn/large/006tKfTcly1g0yx7qlr4dj30rw0eejux.jpg)

### RMSProp

RMSProp 算法修改 AdaGrad 以在非凸设定下效果更好，改变梯度积累为指数加权的移动平均。AdaGrad 旨在应用于凸问题时快速收敛。当应用于非凸函数训练神经网络时，学习轨迹可能穿过了很多不同的结构，最终到达一个局部是凸碗的区域。AdaGrad 根据平方梯度的整个历史收缩学习率，可能使得学习率在达到这样的凸结构前就变得太小了。RMSProp 使用指数衰减平均以丢弃遥远过去的历史，使其能够在找到凸碗状结构后快速收敛，它就像一个初始化于该碗状结构的 AdaGrad 算法实例。

![rms](https://ws3.sinaimg.cn/large/006tKfTcgy1g0yx8jevlhj30k00aadhn.jpg)

### Adam

首先，在 Adam 中，动量直接并入了梯度一阶矩(指数加权)的估计。将动量加入 RMSProp 最直观的方法是将动量应用于缩放后的梯度。结合缩放的动量使用没有明确的理论动机。其次，Adam 包括偏置修正，修正从原点初始化的一阶矩(动量项)和(非中心的)二阶矩的估计(算法 8.7 )。RMSProp 也采用了(非中心的)二阶矩估计，然而缺失了修正因子。因此，不像 Adam，RMSProp 二阶矩估计可能在训练初期有很高的偏置。Adam 通常被认为对超参数的选择相当鲁棒，尽管学习率有时需要从建议的默认修改。

![adam](https://ws1.sinaimg.cn/large/006tKfTcgy1g0yx99w3ubj30k00fj41c.jpg)